### Importar librerías

In [ ]:
import os
import numpy as np
import datetime
import sklearn 

# importar fichero con utilidades propias
from commons import myfunctions as myfunc


#### Definición de parámetros

Se indican cuales son los algoritmos que mayor estabilidad tienen con la utilización de muestras reales

In [ ]:
# parametros
SEMILLA = 42
B_TIPO= "bin_s"
M_TIPO= "mul_s"
DATA_TIPO= "mul_s"
PRE_DATA_FILE=  "rows_transpose_norm_by_gene_id_with_target_num_"

modelo_binario="modelo_"+str(SEMILLA).zfill(4)+"_"+B_TIPO+"_ANOVA_LR_L2.pkl"
modelo_multi= "modelo_"+str(SEMILLA).zfill(4)+"_"+M_TIPO+"_ANOVA_LR_L2.pkl"


Nombre del fichero con las muestras a utilizar

In [ ]:
DATA_FILE= PRE_DATA_FILE + DATA_TIPO


#### Incializar variables, comprobar entorno y leer el fichero con las muestras.

In [ ]:
start_time = datetime.datetime.now()

myfunc.reset_vars()

myfunc.NOTEBK_FILENAME = myfunc.get_nb_name()
myfunc.EXEC_DIR = os.path.join(myfunc.EXEC_DIR, M_TIPO)

myfunc.check_enviroment(myfunc.DATA_DIR, myfunc.CFDNA_DIR, myfunc.GENCODE_DIR, myfunc.H5_DIR, myfunc.LOG_DIR, myfunc.CSV_DIR, myfunc.MODEL_DIR, myfunc.EXEC_DIR, myfunc.MET_DIR)

def mutual_info_classif_state(X, y):
    return mutual_info_classif(X, y, random_state=SEMILLA)

#  leer fichero de datos
df_t = myfunc.read_h5_to_df(DATA_FILE, myfunc.H5_DIR)
display(df_t.groupby("target").size())
print("Shape df:",df_t.shape)


Se separan las muestras utilizando la misma semilla que se utilizó para el entrenamiento del modelo.

Se utilizan las muestras de _test_ para evaluar el rendimiento del modelo.

In [ ]:
from sklearn.model_selection import  train_test_split

X = df_t.iloc[:, :-1]
y = df_t.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

y_test_bin = y_test.copy()
y_test_bin[y_test_bin != 0] = 1

# y_train_bin = y_train.copy()
# y_train_bin[y_train_bin != 0] = 1

print("Shape X_test:",X_test.shape)


#### Leer modelos guardados de clasificación binaria y multiclase indicados en los parámetros

In [ ]:
mejor_modelo_bin=myfunc.read_modelo(myfunc.MODEL_DIR, modelo_binario)

mejor_modelo_mul=myfunc.read_modelo(myfunc.MODEL_DIR, modelo_multi)


### Ver rendimiento en clasificación binaria

In [ ]:
import sklearn

# se obtienen las predicciones de la clasificación binaria
predicciones_bin  = mejor_modelo_bin.predict(X_test)

print(sklearn.metrics.classification_report(y_test_bin, predicciones_bin))

conf_matrix = sklearn.metrics.confusion_matrix(y_test_bin, predicciones_bin)

accuracy = sklearn.metrics.accuracy_score(y_test_bin, predicciones_bin)
precision = sklearn.metrics.precision_score(y_test_bin, predicciones_bin, average="macro")
recall = sklearn.metrics.recall_score(y_test_bin, predicciones_bin, average="macro")
f1 = sklearn.metrics.f1_score(y_test_bin, predicciones_bin, average="macro")

print("\n")
print("           Exactitud: %.6f" % (accuracy)) 
print("   Precisión (media): %.6f" % (precision))
print("      Recall (media): %.6f" % (recall))
print("    F1-score (media): %.6f" % (f1))

myfunc.ver_metricas_bin_matriz_confusion(y_test_bin, predicciones_bin)

### Ver rendimiento con el sistema de clasificación en dos fases y un modelo por fase

Se define el *sistema* con la función *predict_pipelines* donde primero se hace la clasificación binaria y a continuación la clasificación del tipo de cáncer.

La función *predict_proba_pipelines* también se ha definido para calcular las probabilidades y mostrar la matriz de confusión.

Se muestran a continuación ambas funciones, pero realmente están incluidas en la función que muestra las métricas *myfunc.ver_metricas_multi_pipeline*.

In [ ]:
def predict_pipelines(modelo_bin, modelo_mul, X):
    # se obtienen las predicciones de la clasificación binaria
    predict_bin  = modelo_bin.predict(X)
    # se obtienen las predicciones de la clasificación multiclase
    predict_mul  = modelo_mul.predict(X)
    # Finalmente se mezclan las muestras que han salido sanas del primer modelo con el resultado 
    # de las muestras de la clasificación multicáncer que indican el tipo de cáncer predicho
    return np.where(predict_bin == 0, 0, predict_mul)


#### Resultados de la ejecución completa del sistema en 2 fases

In [ ]:

y_pred = predict_pipelines(mejor_modelo_bin, mejor_modelo_mul,X_test)

print(sklearn.metrics.classification_report(y_test, y_pred, zero_division=0, digits=3))

conf_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)

accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred, average="macro", zero_division=0)
recall = sklearn.metrics.recall_score(y_test, y_pred, average="macro", zero_division=0)
f1 = sklearn.metrics.f1_score(y_test, y_pred, average="macro", zero_division=0)

print("\n")
print("           Exactitud: %.6f" % (accuracy)) 
print("   Precisión (media): %.6f" % (precision))
print("      Recall (media): %.6f" % (recall))
print("    F1-score (media): %.6f" % (f1))

myfunc.ver_metricas_multi_matriz_confusion(y_test,y_pred)

#### Finalización del notebook

In [ ]:
end_time = datetime.datetime.now()
total_time = end_time - start_time
myfunc.verbose(f"Notebook ha tardado {total_time} segundos")